In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')      
# !pip install transformers

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict
from datetime import datetime
import fnmatch
import itertools
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import RandomState
import os
from pdb import set_trace
import pandas as pd
import pickle
import pprint
import random
from sklearn.linear_model import SGDClassifier 
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, auc, precision_recall_curve
from sklearn.metrics import precision_recall_fscore_support as score
import seaborn as sns
import warnings
import torch
import uuid
import time
import copy

import sys
sys.path.append("/content/drive/My Drive/collab/TADAT/") 
#local
from tadat.pipeline import plots
import tadat.core as core


warnings.filterwarnings("ignore")
sns.set(style="whitegrid")

## Configs

In [3]:

# BASE_PATH = "/content/drive/My Drive/collab/MIMIC/"
BASE_PATH = "/Users/samir/Dev/projects/MIMIC/MIMIC/"
INPUT_PATH = BASE_PATH+"/DATA/input/"
FEATURES_PATH = BASE_PATH+"/DATA/features/"
OUTPUT_PATH = BASE_PATH+"/DATA/results/"
TMP_PATH = BASE_PATH+"/DATA/processed/"

TUNE_OUTPUT_PATH = BASE_PATH+"/DATA/results_fine/"
TUNE_TMP_PATH = BASE_PATH+"/DATA/processed_fine/"

GRID_OUTPUT_PATH = BASE_PATH+"/DATA/results_grid/"
GRID_TMP_PATH = BASE_PATH+"/DATA/processed_grid/"

#configs
N_SEEDS=4
N_VAL_SEEDS = 5
N_VAL_RUNS = 5
N_TASKS = 3
N_TASKS = 50
# PLOT_VARS=["auroc","auprc","sensitivity","specificity"]
PLOT_VARS=["auroc","sensitivity"]
MODEL="BERT-POOL"
METRIC = "auroc"

GROUPS = { "GENDER": ["M","F"],   
         "ETHNICITY": ["WHITE","BLACK","ASIAN","HISPANIC"]
}

CLASSIFIER = 'sklearn'
CLASSIFIER = 'torch'
# CLASSIFIER = 'mseq'
CLINICALBERT = "emilyalsentzer/Bio_ClinicalBERT"

In [4]:

SMALL_SIZE = 18
MEDIUM_SIZE = 20
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('text', usetex = True)

# Modeling 

In [5]:
def train_classifier(X_train, Y_train, X_val, Y_val, 
                     init_seed, shuffle_seed=None, input_dimension=None):    
    """ train a classifier
        X_train: training instances 
        Y_yrain: training labels
        X_val: validation instances
        Y_val: validation labels
        init_seed: parameter initialization seed
        shuffle_seed: data shuffling seed
        input_dimension: number of input features
        
        returns: fitted classifier
    """
    #CLASSIFIER is a global variable indicating the type of classifier
    if CLASSIFIER == "torch":        
        x = core.models.MyLinearModel(in_dim=input_dimension, out_dim=1, 
                    loss_fn=torch.nn.BCEWithLogitsLoss(), 
                    init_seed=init_seed, n_epochs=500, 
                    default_lr=0.1, batch_size=None, 
                    shuffle_seed=shuffle_seed, silent=True,
                    shuffle=True) 
        x.fit(X_train, Y_train, X_val, Y_val)
    elif CLASSIFIER == "mseq":        
        x = core.models.MultiSeqLinearModel(in_dim=input_dimension, out_dim=1, 
                    loss_fn=torch.nn.BCELoss(), 
                    init_seed=init_seed, n_epochs=500, 
                    default_lr=0.1, batch_size=None, 
                    shuffle_seed=shuffle_seed, silent=True,
                    shuffle=True) 
        x.fit(X_train, Y_train, X_val, Y_val)
    elif CLASSIFIER == "sklearn":
        x = SGDClassifier(loss="log", random_state=shuffle_seed)
        x.fit(X_train, Y_train)
    else:
        raise NotImplementedError
    return x

def evaluate_classifier(model, X_test, Y_test,
                   labels, model_name, random_seed, subgroup, res_path=None):
    """ evaluate a classifier
        model: classifier to be evaluated        
        X_test: test instances
        Y_test: test labels
        labels: label set
        model_name: model name
        random_seed: random seed that was used to train the classifier
        subgroup: demographic subgroup represented in the data
        res_path: path to save the results
        
        returns: dictionary of evaluation wrt to different metrics
    """
    Y_hat = model.predict(X_test)
    Y_hat_prob = model.predict_proba(X_test)
    #get probabilities for the positive class
    if CLASSIFIER == 'sklearn':
        Y_hat_prob = Y_hat_prob[:,labels[1]]    
    microF1 = f1_score(Y_test, Y_hat, average="micro") 
    macroF1 = f1_score(Y_test, Y_hat, average="macro") 
    try:
        aurocc = roc_auc_score(Y_test, Y_hat_prob)
    except ValueError:
        aurocc = 0
    try:
        prec, rec, thresholds = precision_recall_curve(Y_test, Y_hat_prob)       
        auprc = auc(rec, prec)
    except ValueError:
        auprc = 0
    try:
        tn, fp, fn, tp = confusion_matrix(Y_test, Y_hat).ravel()
        specificity = tn / (tn+fp)
        sensitivity = tp / (fn+tp)
    except ValueError:
        specificity, sensitivity = 0, 0
    
    res = {"model":model_name, 
            "seed":random_seed,  
            "group":subgroup,    
            "microF1":round(microF1,3),
            "macroF1":round(macroF1,3),
            "auroc":round(aurocc,3),
            "auprc":round(auprc,3),
            "specificity":round(specificity,3),
            "sensitivity":round(sensitivity,3)           
            }

    if res_path is not None:    
        core.helpers.save_results(res, res_path, sep="\t")
    return res



def vectorize(df_train, df_val, df_test, subject_ids):
    """ vectorize the instances and stratify them by demographic subgroup
        df_train: training data as a DataFrame
        df_test: test data as a DataFrame
        df_val: validation data as a DataFrame
        subject_ids: list of subject ids (the order corresponds to order of the features that were extracted)
        
        returns: vectorized train, validation and test datasets, stratified by demographic subgroup
                 label vocabulary                 
    """

    #vectorize labels
    train_Y = df_train["Y"]
    val_Y = df_val["Y"]           
    test_Y = df_test["Y"]               
    label_vocab = core.vectorizer.get_labels_vocab(train_Y+val_Y)
    train_Y,_ = core.vectorizer.label2idx(train_Y, label_vocab)
    val_Y,_ = core.vectorizer.label2idx(val_Y, label_vocab)
    test_Y,_ = core.vectorizer.label2idx(test_Y, label_vocab)      
    
    #get indices into the feature matrix
    train_idxs = [subject_ids.index(i) for i in list(df_train["SUBJECT_ID"])] 
    val_idxs = [subject_ids.index(i) for i in list(df_val["SUBJECT_ID"])] 
    test_idxs = [subject_ids.index(i) for i in list(df_test["SUBJECT_ID"])] 
    #construct datasets
    train = {}
    test = {}
    val = {}
    #unstratified 
    train["all"] = [train_idxs, train_Y]
    test["all"] = [test_idxs, test_Y]
    val["all"] = [val_idxs, val_Y]
    #stratified by demographics 
    for group in list(GROUPS.keys()):
        #and subgroups
        for subgroup in GROUPS[group]:                
            df_train_sub = df_train[df_train[group] == subgroup]
            df_test_sub = df_test[df_test[group] == subgroup]
            df_val_sub = df_val[df_val[group] == subgroup]
            # print("[subgroup: {} | tr: {} | ts: {} | val: {}]".format(subgroup, len(df_train_sub), len(df_test_sub), len(df_val_sub)))

            #vectorize labels               
            train_Y_sub,_ = core.vectorizer.label2idx(df_train_sub["Y"], label_vocab)            
            test_Y_sub,_ = core.vectorizer.label2idx(df_test_sub["Y"], label_vocab)            
            val_Y_sub,_ = core.vectorizer.label2idx(df_val_sub["Y"], label_vocab)      
            #get indices into the feature matrix
            train_idxs_sub = [subject_ids.index(i) for i in list(df_train_sub["SUBJECT_ID"])] 
            test_idxs_sub = [subject_ids.index(i) for i in list(df_test_sub["SUBJECT_ID"])] 
            val_idxs_sub = [subject_ids.index(i) for i in list(df_val_sub["SUBJECT_ID"])] 
            if subgroup == "M":
                subgroup = "men"
            elif subgroup == "F":
                subgroup = "women"
            train[subgroup.lower()] = [train_idxs_sub, train_Y_sub]
            test[subgroup.lower()] = [test_idxs_sub, test_Y_sub]
            val[subgroup.lower()] = [val_idxs_sub, val_Y_sub]

    return train, val, test, label_vocab


def get_features(data, vocab_size, feature_type, word_vectors=None):
    """ compute features from the data
        data: data instances
        vocab_size: size of the vocabulary
        feature_type: type of feature (e.g bag of words, BERT)
        word_vectors: path to pretrained (static) word vectors
        
        returns: feature matrix
    """
    if feature_type == "BOW-BIN":
        X = core.features.BOW(data, vocab_size,sparse=True)
    elif feature_type == "BOW-FREQ":
        X = core.features.BOW_freq(data, vocab_size,sparse=True)
    elif feature_type == "BOE-BIN":
        X = core.features.BOE(data, word_vectors,"bin")
    elif feature_type == "BOE-SUM": 
        X = core.features.BOE(data, word_vectors,"sum")
    elif feature_type == "BERT-POOL":
        X =  core.transformer_encoders.encode_sequences(data, batchsize=64)        
    elif feature_type == "BERT-CLS":
        X =  core.transformer_encoders.encode_sequences(data, cls_features=True,
                                                        batchsize=64)            
    elif feature_type == "MULTI-BERT-POOL":
        X =  core.transformer_encoders.encode_multi_sequences(data, 10, batchsize=32,
                                                         tmp_path=TMP_PATH)
    elif feature_type == "MULTI-BERT-CLS":
        X =  core.transformer_encoders.encode_multi_sequences(data, 10, 
                                                         cls_features=True,
                                                         batchsize=32,
                                                         tmp_path=TMP_PATH)
    elif feature_type == "CLINICALBERT-POOL":
        tokenizer, encoder = core.transformer_encoders.get_encoder(CLINICALBERT)
        X =  core.transformer_encoders.encode_sequences(data, batchsize=64, tokenizer=tokenizer,
                                                                    encoder=encoder)        
    elif feature_type == "CLINICALBERT-CLS":
        tokenizer, encoder = core.transformer_encoders.get_encoder(CLINICALBERT)
        X =  core.transformer_encoders.encode_sequences(data, cls_features=True,batchsize=64,
                                                                    tokenizer=tokenizer, encoder=encoder)        
    elif feature_type == "CLINICALMULTI-BERT-POOL":
        tokenizer, encoder = core.transformer_encoders.get_encoder(CLINICALBERT)
        X =  core.transformer_encoders.encode_multi_sequences(data, 10, batchsize=32,tmp_path=TMP_PATH,
                                                              tokenizer=tokenizer, encoder=encoder)
    elif feature_type == "CLINICALMULTI-BERT-CLS":
        tokenizer, encoder = core.transformer_encoders.get_encoder(CLINICALBERT)
        X =  core.transformer_encoders.encode_multi_sequences(data, 10, cls_features=True, 
                                                                batchsize=32,tmp_path=TMP_PATH,
                                                                tokenizer=tokenizer, encoder=encoder)
    else:
        raise NotImplementedError
    return X

def extract_features(feature_type, path):
    """ extract features and save features

        method will first look for computed features on disk and return them if found;
        otherwise it will look for a file with name *patients.csv*;        
        
        feature_type: type of feature (e.g bag of words, BERT)
        path: directory where the data can be found
                
        returns: list of subject ids and feature matrix -- the order of ids corresponds to order of the instances in the feature matrix
    """
    X = read_cache(path+"feats_{}".format(feature_type))
    if X:
        print("[reading cached features]")
        subject_ids, X_feats = X
    else:
        print("[computing {} features]".format(feature_type))
        df = pd.read_csv(path+"patients.csv", sep="\t", header=0)
        subject_ids = list(df["SUBJECT_ID"])
        docs = list(df["TEXT"])
        if "BERT" in feature_type:
            X_feats = get_features(docs, None, feature_type)
        else:
            X, word_vocab = core.vectorizer.docs2idx(docs)
            X_feats = get_features(X,len(word_vocab),feature_type)
        #save features
        print("[saving features]")
        write_cache(path+"feats_{}".format(feature_type), 
                    [subject_ids, X_feats])
    return subject_ids, X_feats



# Run

In [6]:
def read_dataset(path, dataset_name, df_patients):    
    
    """ read dataset        
        path: path to the dataset
        dataset_name: name of the dataset
        df_patients: DataFrame of patients
                
        returns: train, test and validation sets as DataFrames
    """
    df_train = pd.read_csv("{}/{}_train.csv".format(path, dataset_name), 
                           sep="\t", header=0)
    df_test  = pd.read_csv("{}/{}_test.csv".format(path, dataset_name),
                           sep="\t", header=0)
    df_val   = pd.read_csv("{}/{}_val.csv".format(path, dataset_name),
                           sep="\t", header=0)
    #set indices
    df_patients.set_index("SUBJECT_ID", inplace=True)
    df_train.set_index("SUBJECT_ID", inplace=True)
    df_test.set_index("SUBJECT_ID", inplace=True)
    df_val.set_index("SUBJECT_ID", inplace=True)

    df_train = df_train.join(df_patients, on="SUBJECT_ID", 
                             how="inner", lsuffix="N_").reset_index()
    df_test = df_test.join(df_patients, on="SUBJECT_ID", 
                           how="inner", lsuffix="N_").reset_index()
    df_val = df_val.join(df_patients, on="SUBJECT_ID", 
                         how="inner", lsuffix="N_").reset_index()

    return df_train, df_test, df_val   


def read_cache(path):
    """ read a pickled object
        
        path: path
        returns: object
    """
    X = None
    try:
        with open(path, "rb") as fi:            
            X = pickle.load(fi)
    except FileNotFoundError:
        pass
    return X

def write_cache(path, o):
    """ pickle an object
            
        path: path
        o: object
    """
    dirname = os.path.dirname(path)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    with open(path, "wb") as fo:
        pickle.dump(o, fo)

def run(data_path, dataset, features_path, feature_type, cache_path, metric, n_seeds=N_SEEDS, clear_results=False):
    """ 
        train classifiers with different random seeds and compare the performance over each demographic subgroup

        data_path: path to the data
        dataset: dataset to be evaluted
        features_path: path to the features
        feature_type: type of feature (e.g bag of words, BERT)
        cache_path: cache path 
        metric: evaluation metric
        n_seeds: number of seeds

        returns: results for each subgroup
    """
    #read patients data
    df_patients = pd.read_csv(features_path+"patients.csv", 
                              sep="\t", header=0).drop(columns=["TEXT"])
    #read dataset
    df_train, df_test, df_val = read_dataset(data_path, dataset, df_patients)
    
    print("[train/test set size: {}/{}]".format(len(df_train), len(df_test)))
    print("[running {} classifier]".format(CLASSIFIER))
    #extract features
    subject_ids, feature_matrix = extract_features(feature_type, features_path)      
    train, val, test, label_vocab = vectorize(df_train, df_val, df_test, subject_ids)
    train_idx, train_Y = train["all"]
    val_idx, val_Y = val["all"]
    #slice the feature matrix to get the corresponding instances
    train_X = feature_matrix[train_idx, :]    
    val_X = feature_matrix[val_idx, :]    
    #create the cache directory if it does not exist
    dirname = os.path.dirname(cache_path)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    #try to open a cached results file or create a new one if it does not exist
    res_fname = cache_path+"/seeds_{}_{}_{}.pkl".format(dataset, feature_type, metric).lower()    
    try:
        df_results = pd.read_csv(res_fname)
    except FileNotFoundError:
        df_results = pd.DataFrame(columns = ["seed"] +  list(val.keys()))
        df_results.to_csv(res_fname, index=False, header=True)        
    #we can skip seeds that have already been evaluated
    skip_seeds = set([]) if clear_results else set(df_results["seed"])
    groups = list(val.keys())
    random.seed(1) #ensure repeateable runs 
    random_seeds = random.sample(range(0, 10000), n_seeds)        
    incremental_results = {}     
    ##train/test classifier for each random seed pair
    for init_seed, shuffle_seed in itertools.product(random_seeds,repeat=2):   
        seed = "{}x{}".format(init_seed, shuffle_seed)          
        if seed in skip_seeds:
            print("skipped seed: {}".format(seed))
            continue
        curr_results = {"seed":seed}
        print(" > seed: {}".format(seed))                        
        model = train_classifier(train_X, train_Y,val_X, val_Y,  
                                    input_dimension=train_X.shape[-1],
                                    init_seed=init_seed, 
                                    shuffle_seed=shuffle_seed)                                                                                
        #test each subgroup (note thtat *all* is also a subgroup)
        for subgroup in groups:                                
            test_idx_sub, test_Y_sub = test[subgroup]                 
            test_X_sub = feature_matrix[test_idx_sub, :]                
            res_sub = evaluate_classifier(model, test_X_sub, test_Y_sub, 
                                        label_vocab, feature_type, seed, subgroup)                
            curr_results[subgroup]= res_sub[metric]     
        #save results
        df_results = df_results.append(curr_results, ignore_index=True)
        df_results.to_csv(res_fname, index=False, header=True)

    return df_results


def subsample_run(data_path, dataset, features_path, feature_type, cache_path, n_draws=10):
    #read patients data
    df_patients = pd.read_csv(features_path+"patients.csv", 
                              sep="\t", header=0).drop(columns=["TEXT"])

    df_train, df_test, df_val = read_dataset(data_path, dataset, df_patients)
    
    print("[train/test set size: {}/{}]".format(len(df_train), len(df_test)))
    print("[{} classifier]".format(CLASSIFIER))
    subject_ids, feature_matrix = extract_features(feature_type, features_path)      
    train, val, test, label_vocab = vectorize(df_train, df_val, df_test, subject_ids)
    train_idx, train_Y = train["all"]
    val_idx, val_Y = val["all"]
    #slice the feature matrix to get the corresponding instances
    train_X = feature_matrix[train_idx, :]    
    val_X = feature_matrix[val_idx, :]    
    random.seed(1) #ensure repeateable runs 
    random_seeds = random.sample(range(0, 10000), N_SEEDS)        
    incremental_results = {}     
    sample_size = min([len(test[subgroup][0]) for subgroup in test.keys()])
    print(sample_size)
    ##train/test classifier for each random seed pair    
    for init_seed, shuffle_seed in itertools.product(random_seeds,repeat=2):        
        seed = "{}x{}".format(init_seed, shuffle_seed)
        res_fname = "{}_{}_res{}.pkl".format(dataset, feature_type, seed).lower()                        
        curr_results = {}
        print(" > seed: {}".format(seed))                        
        model = train_classifier(train_X, train_Y,val_X, val_Y,  
                                    input_dimension=train_X.shape[-1],
                                    init_seed=init_seed, 
                                    shuffle_seed=shuffle_seed)                                                      
        for i in range(n_draws):
            #evaluate different random samples of the data
            #test each subgroup (note thtat *all* is also a subgroup)            
            for subgroup in test.keys():                                
                test_idx_sub, test_Y_sub = test[subgroup]            
                test_Y_sub = np.array(test_Y_sub)
                test_idx_sub = np.array(test_idx_sub)                    
                random_sample = random.sample(range(len(test_idx_sub)), sample_size)                
                test_Y_sub_sample = test_Y_sub[random_sample]
                test_idx_sub_sample = test_idx_sub[random_sample]                    
                test_X_sub_sample = feature_matrix[test_idx_sub_sample, :]                
                res_sub = evaluate_classifier(model, test_X_sub_sample, test_Y_sub_sample, 
                                            label_vocab, feature_type, seed+"x"+str(i), subgroup)                
                curr_results[subgroup]= res_sub                       
        
            incremental_results = merge_results(curr_results, incremental_results, 
                                            list(test.keys()))
    #build dataframes 
    df_results = results_to_df(incremental_results, test.keys())
    return df_results


# Validation vs Test Performance

In [11]:
# Here we investigate the correlation between validation and test performance as a function of the random seed

def val_vs_test(data_path, dataset, features_path, feature_type, cache_path, metric, n_seeds=N_SEEDS):
    """ 
        compare the validation and test performance of classifiers as a function of the random seeds
        
        data_path: path to the data
        dataset: dataset to be evaluted
        features_path: path to the features
        feature_type: type of feature (e.g bag of words, BERT)
        cache_path: cache path 
        metric: evaluation metric 
        n_seeds: number of random seeds to be evaluated 

        returns: results for each demographic group
    """
    #read patients data
    df_patients = pd.read_csv(features_path+"patients.csv", 
                              sep="\t", header=0).drop(columns=["TEXT"])

    df_train, df_test, df_val = read_dataset(data_path, dataset, df_patients)
    
    print("[train/test set size: {}/{}]".format(len(df_train), len(df_test)))
    print("[running {} classifier]".format(CLASSIFIER))
    subject_ids, feature_matrix = extract_features(feature_type, features_path)      
    train, val, test, label_vocab = vectorize(df_train, df_val, df_test, subject_ids)
    train_idx, train_Y = train["all"]
    val_idx, val_Y = val["all"]
    #slice the feature matrix to get the corresponding instances
    train_X = feature_matrix[train_idx, :]    
    val_X = feature_matrix[val_idx, :]    
    
    dirname = os.path.dirname(cache_path)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    res_fname = cache_path+"/val_vs_test_{}_{}_{}.pkl".format(dataset, feature_type, metric).lower()    
    try:
        df_results = pd.read_csv(res_fname)
    except FileNotFoundError:
        df_results = pd.DataFrame(columns = ["seed","data"] +  list(val.keys()))
        df_results.to_csv(res_fname, index=False, header=True)        
    groups = list(val.keys())
    skip_seeds = set(df_results["seed"])

    random.seed(1) #ensure repeateable runs 
    random_seeds = random.sample(range(0, 10000), n_seeds)            
    ##train/test classifier for each random seed pair
    for init_seed, shuffle_seed in itertools.product(random_seeds,repeat=2):        
        seed = "{}x{}".format(init_seed, shuffle_seed)          
        if seed in skip_seeds:
            print("skipped seed: {}".format(seed))
            continue
        test_results = {"seed":seed, "data":"test"}
        print(" > seed: {}".format(seed))                        
        model = train_classifier(train_X, train_Y,val_X, val_Y,  
                                    input_dimension=train_X.shape[-1],
                                    init_seed=init_seed, 
                                    shuffle_seed=shuffle_seed)                                                                                
        #test each subgroup on test data (note thtat *all* is also a subgroup)
        for subgroup in groups:                                
            test_idx_sub, test_Y_sub = test[subgroup]                 
            test_X_sub = feature_matrix[test_idx_sub, :]                
            res_sub = evaluate_classifier(model, test_X_sub, test_Y_sub, 
                                        label_vocab, feature_type, seed, subgroup)                
            test_results[subgroup]= res_sub[metric]     
        df_results = df_results.append(test_results, ignore_index=True)
        val_results = {"seed":seed, "data":"val"}
        #test each subgroup on validation data (note thtat *all* is also a subgroup)
        for subgroup in groups:                                
            val_idx_sub, val_Y_sub = val[subgroup]                 
            val_X_sub = feature_matrix[val_idx_sub, :]                
            res_sub = evaluate_classifier(model, val_X_sub, val_Y_sub, 
                                        label_vocab, feature_type, seed, subgroup)                
            val_results[subgroup]= res_sub[metric]     

        df_results = df_results.append(val_results, ignore_index=True)
        df_results.to_csv(res_fname, index=False, header=True)

    return df_results

def all_val_vs_test(data_path, tasks_fname, features_path, feature_type, cache_path, metric, n_seeds=N_SEEDS):
    """ 
        compare the validation and test performance of classifiers as a function of the random seeds for all the datasets
        
        data_path: path to the data
        dataset: dataset to be evaluted
        features_path: path to the features
        feature_type: type of feature (e.g bag of words, BERT)
        cache_path: cache path 
        metric: evaluation metric 
        n_seeds: number of random seeds to be evaluated 
    """
    with open(data_path+"/"+tasks_fname,"r") as fid:        
        for i,l in enumerate(fid):            
            task_abv, task_name = l.strip("\n").split(",")        
            print(task_name+"\n")    
            val_vs_test(data_path, "mini-"+task_abv, features_path, feature_type, cache_path, metric, n_seeds=n_seeds)


def plot_val_vs_test(cache_path, dataset, feature_type, metric):
    res_fname = cache_path+"/val_vs_test_{}_{}_{}.pkl".format(dataset, feature_type, metric).lower()    
    df_results = pd.read_csv(res_fname)  
    df_val = df_results[df_results["data"]=="val"]
    df_test = df_results[df_results["data"]=="test"]
    subgroups = ["men","women","white","black","hispanic","asian"]
    n_rows=2
    n_cols = 3
    fig, ax = plt.subplots(n_rows, n_cols,  figsize=(15,10), sharex=True, sharey=True)
    #current coloramap
    cmap = plt.rcParams['axes.prop_cycle'].by_key()['color']
    coords = list(itertools.product(range(n_rows),range(n_cols)))   
    
    for subgroup, col, coord in zip(subgroups, cmap, coords ):                
        cax = ax[coord[0]][coord[1]]
        cax.scatter(x=df_val[subgroup],y=df_test[subgroup],
                            color=col)                
        x = df_val[subgroup]
        y = df_test[subgroup]
        z = np.polyfit(x, y, 1)
        y_hat = np.poly1d(z)(x)
        ax[coord[0]][coord[1]].plot(x, y_hat, c=col, lw=1)
        ax[coord[0]][coord[1]].set_title(subgroup)        
        ax[coord[0]][coord[1]].set_xlabel("val")
        ax[coord[0]][coord[1]].set_ylabel("test")
    fig.suptitle("val vs test", y=1.02)
    plt.tight_layout()
    plt.show()

def plot_val_vs_test_deltas(cache_path, dataset, feature_type, metric):
    res_fname = cache_path+"/val_vs_test_{}_{}_{}.pkl".format(dataset, feature_type, metric).lower()    
    df_results = pd.read_csv(res_fname)  
    df_val = df_results[df_results["data"]=="val"]
    df_test = df_results[df_results["data"]=="test"]    
    subgroups = ["men","women","white","black","hispanic","asian"]
    df_val_all = df_val["all"]
    df_test_all = df_test["all"]
    for s in subgroups:
        df_val["delta_"+s] = (df_val[s] - df_val_all) #.abs()
        df_test["delta_"+s] = (df_test[s] - df_test_all) #.abs()

    n_rows=2
    n_cols = 3
    fig, ax = plt.subplots(n_rows, n_cols,  figsize=(15,10), sharex=True, sharey=True)
    #current coloramap
    cmap = plt.rcParams['axes.prop_cycle'].by_key()['color']
    coords = list(itertools.product(range(n_rows),range(n_cols)))   
    
    for subgroup, col, coord in zip(subgroups, cmap, coords ):                
        cax = ax[coord[0]][coord[1]]
        cax.scatter(x=df_val["delta_"+subgroup],y=df_test["delta_"+subgroup],
                            color=col)                
        x = df_val["delta_"+subgroup]
        y = df_test["delta_"+subgroup]
        z = np.polyfit(x, y, 1)
        y_hat = np.poly1d(z)(x)
        ax[coord[0]][coord[1]].plot(x, y_hat, c=col, lw=1)
        ax[coord[0]][coord[1]].set_title(subgroup)        
        ax[coord[0]][coord[1]].set_xlabel("val")
        ax[coord[0]][coord[1]].set_ylabel("test")
    fig.suptitle("val vs test", y=1.02)
    plt.tight_layout()
    plt.show()


def plot_all_val_vs_test(data_path, tasks_fname, features_path, feature_type, cache_path, metric, deltas=True):
    with open(data_path+"/"+tasks_fname,"r") as fid:        
        for i,l in enumerate(fid):            
            task_abv, task_name = l.strip("\n").split(",")        
            print(task_name+"\n")
            if deltas:
                plot_val_vs_test_deltas(cache_path, task_abv, feature_type, metric)
            else:
                plot_val_vs_test(cache_path, task_abv, feature_type, metric)

In [12]:
dataset="mini-IHM"
# z = val_vs_test(INPUT_PATH, dataset, FEATURES_PATH, MODEL, TMP_PATH, METRIC, n_seeds=N_SEEDS)
all_val_vs_test(INPUT_PATH, "tasks.txt", FEATURES_PATH, MODEL, TMP_PATH, METRIC, n_seeds=25)
# MODEL="BERT-POOL"
plot_all_val_vs_test(INPUT_PATH, "tasks.txt", FEATURES_PATH, MODEL, TMP_PATH, METRIC)
# plot_all_val_vs_test(INPUT_PATH, "tasks.txt", FEATURES_PATH, MODEL, TMP_PATH, METRIC, deltas=False)

In Hospital Mortality

[train/test set size: 1000/1000]
[running torch classifier]
[reading cached features]
skipped seed: 2201x2201
skipped seed: 2201x9325
skipped seed: 2201x1033
skipped seed: 2201x4179
 > seed: 2201x1931
[early stopping: 148 epochs]
 > seed: 2201x8117
[early stopping: 115 epochs]
 > seed: 2201x7364
[early stopping: 145 epochs]
 > seed: 2201x7737
[early stopping: 109 epochs]
 > seed: 2201x6219
[early stopping: 146 epochs]
 > seed: 2201x3439
[early stopping: 153 epochs]
 > seed: 2201x1537
[early stopping: 114 epochs]
 > seed: 2201x7993
[early stopping: 104 epochs]
 > seed: 2201x464
[early stopping: 111 epochs]
 > seed: 2201x6386
[early stopping: 108 epochs]
 > seed: 2201x7090
[early stopping: 104 epochs]
 > seed: 2201x9952
[early stopping: 117 epochs]
 > seed: 2201x34
[early stopping: 116 epochs]
 > seed: 2201x7297
[early stopping: 119 epochs]
 > seed: 2201x4363
[early stopping: 118 epochs]
 > seed: 2201x3748
[early stopping: 145 epochs]
 > seed: 2201x9685
[early stop

KeyboardInterrupt: 

# Data Maps

In [32]:
from sklearn.decomposition import PCA

def data_mapper(data_path, dataset, features_path, feature_type, cache_path, 
                shuffle_seed=1,init_seed=2, update_cache=False):
    seed = "{}x{}".format(init_seed, shuffle_seed)
    res_fname = cache_path+"/datamap_{}_{}_{}.pkl".format(dataset, feature_type, seed).lower()    
    res = read_cache(res_fname)    
    #read patients data
    df_patients = pd.read_csv(features_path+"patients.csv", 
                                sep="\t", header=0).drop(columns=["TEXT"])    
    subject_ids, feature_matrix = extract_features(feature_type, features_path)      
    
    if not res or update_cache:
        df_train, df_test, df_val = read_dataset(data_path, dataset, df_patients)
        print("[train/test set size: {}/{}]".format(len(df_train), len(df_test)))
        train, val, test, label_vocab = vectorize(df_train, df_val, df_test, subject_ids)
        train_idx, train_Y = train["all"]
        val_idx, val_Y = val["all"]
        #get a subset of the data only
        max_instances = 12000
        train_idx = train_idx[:max_instances]
        train_Y = train_Y[:max_instances]
        #slice the feature matrix to get the corresponding instances
        train_X = feature_matrix[train_idx, :]    
        val_X = feature_matrix[val_idx, :]    
        input_dimension = train_X.shape[-1]        
        model = core.models.LinearDataMaps(in_dim=input_dimension, out_dim=1, 
                            loss_fn=torch.nn.BCEWithLogitsLoss(), 
                            init_seed=init_seed, n_epochs=300, 
                            default_lr=0.1, batch_size=None, 
                            shuffle_seed=shuffle_seed, silent=True,
                            shuffle=True) 
        tr_losses, val_losses, tr_preds = model.fit(train_X, train_Y, val_X, val_Y)
        Y = np.array(train_Y).reshape(-1,1)
        write_cache(res_fname, [train_idx, tr_preds, Y])
    else:
        train_idx, tr_preds, Y = res
        print("found cached results")
    #get probability of correct class
    threshold = 0.5
    tr_probs = np.abs((1-Y)-tr_preds)
    confidence = tr_probs.mean(axis=1) + (np.random.random(len(tr_probs)) * 0.01)
    variability = tr_probs.std(axis=1) + (np.random.random(len(tr_probs)) * 0.01)
    correctness = np.where(tr_probs >= threshold, 1, 0).mean(axis=1) + (np.random.random(len(tr_probs)) * 0.01)   
    
    fig2, ax2 = plt.subplots(1, 3,  figsize=(30,5))
    paleta = "BuRd"
    sns.scatterplot(ax=ax2[0], x=variability, y=confidence, hue=correctness, palette=paleta)
    ax2[0].set_title("{} (seed: {})".format(dataset, seed))    
    train_X = feature_matrix[train_idx, :]    
    pca = PCA(n_components=2)
    train_z = pca.fit_transform(train_X)    
    sns.scatterplot(ax=ax2[1], x=train_z[:,0], y=train_z[:,1], hue=confidence, palette=paleta)
    sns.scatterplot(ax=ax2[2], x=train_z[:,0], y=train_z[:,1], hue=variability, palette=paleta)
    ax2[1].set_title("confidence")
    ax2[2].set_title("variability")    
    plt.tight_layout()
    plt.show()
    return tr_preds, Y

def datamap_tasks(data_path, tasks_fname, features_path, feature_type, cache_path, mini_tasks=True):
    with open(data_path+tasks_fname,"r") as fid:        
        for i,l in enumerate(fid):            
            task_abv, task_name = l.strip("\n").split(",")
            dataset = "mini-"+task_abv if mini_tasks else task_abv
            data_mapper(data_path, dataset, features_path, feature_type, cache_path, update_cache=False)
            print("")

In [33]:
# MODEL="CLINICALBERT-POOL"
# dataset="CD"
# tr_preds, Y = data_mapper(INPUT_PATH, dataset, FEATURES_PATH, MODEL, TMP_PATH, update_cache=False)
# datamap_tasks(INPUT_PATH, "tasks.txt", FEATURES_PATH, MODEL, TMP_PATH, mini_tasks=False)